In [0]:
"""Notebook MVP 04 - Gerar tabelas Gold de conformidade por ciclo e por periodo.

Entrada:
- silver_ciclos_referencia.parquet: 1 linha por Data x Ciclo x KM x Recurso,
  com colunas derivadas do racional de ciclos e `Minuto_no_Ciclo` calculado.

Saidas:
- gold_conformidade_ciclo_km.parquet: 1 linha por Mes x KM x Ciclo x Periodo,
  com media da posicao no ciclo, saldo e status por ciclo.
- gold_conformidade_periodo_km.parquet: 1 linha por Mes x KM x Periodo,
  com saldo acumulado do periodo (soma dos 4 ciclos) e status do periodo.

Obs.: Mes sera representado como string AAAA-MM para simplificar.
"""

import os
from pathlib import Path

import numpy as np
import pandas as pd

WORKSPACE_BASE = "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp"
SILVER_DIR = f"{WORKSPACE_BASE}/silver"
GOLD_DIR = f"{WORKSPACE_BASE}/gold"

ARQ_SILVER_REF = f"{SILVER_DIR}/silver_ciclos_referencia.parquet"
ARQ_GOLD_CICLO = f"{GOLD_DIR}/gold_conformidade_ciclo_km.parquet"
ARQ_GOLD_PERIODO = f"{GOLD_DIR}/gold_conformidade_periodo_km.parquet"
ARQ_GOLD_CICLO_XLSX = f"{GOLD_DIR}/gold_conformidade_ciclo_km.xlsx"
ARQ_GOLD_PERIODO_XLSX = f"{GOLD_DIR}/gold_conformidade_periodo_km.xlsx"

LIMITE_INDIVIDUAL = 180.0  # minutos por ciclo
LIMITE_GERAL = 60.0        # minutos por periodo

print("=== CONFIGURACAO DE CAMINHOS ===")
print(f"Workspace base: {WORKSPACE_BASE}")
print(f"Diretorio Silver: {SILVER_DIR}")
print(f"Diretorio Gold: {GOLD_DIR}")
print(f"\nArquivo silver referencia: {ARQ_SILVER_REF}")
print(f"Saida Gold (ciclo): {ARQ_GOLD_CICLO}")
print(f"Saida Gold (periodo): {ARQ_GOLD_PERIODO}")
print(f"Saida Excel ciclo: {ARQ_GOLD_CICLO_XLSX}")
print(f"Saida Excel periodo: {ARQ_GOLD_PERIODO_XLSX}")

print("\n=== CONFIGURACAO DE LIMITES ===")
print(f"Limite individual (ciclo): {LIMITE_INDIVIDUAL} minutos")
print(f"Limite geral (periodo): {LIMITE_GERAL} minutos")

# Criar diretorio gold se nao existir
os.makedirs(GOLD_DIR, exist_ok=True)
print(f"\n[OK] Diretorio Gold criado/verificado: {GOLD_DIR}")

# Verificar se arquivo existe
if not os.path.exists(ARQ_SILVER_REF):
    raise FileNotFoundError(
        f"silver_ciclos_referencia nao encontrado: {ARQ_SILVER_REF}\n"
        "Execute primeiro o notebook 02_gerar_silver_ciclos_referencia"
    )

print("\n=== CARREGANDO DADOS SILVER REFERENCIA ===")
print(f"Lendo silver_ciclos_referencia de: {ARQ_SILVER_REF}")
df_ref = pd.read_parquet(ARQ_SILVER_REF)
print(f"[OK] Linhas carregadas: {len(df_ref):,}")
print(f"Colunas: {list(df_ref.columns)}")

if "Minuto_no_Ciclo" not in df_ref.columns:
    raise ValueError("Coluna Minuto_no_Ciclo nao encontrada em silver_ciclos_referencia.")

print("\n=== PROCESSANDO DADOS ===")
print("Criando coluna Mes (AAAA-MM)...")
df_ref["Mes"] = pd.to_datetime(df_ref["Data"]).dt.to_period("M").astype(str)

print("Definindo Periodo com base no ciclo (1-4 = P1, 5-8 = P2)...")
df_ref["Periodo"] = np.where(df_ref["Ciclo"] <= 4, "P1", "P2")

print(f"\nDistribuicao por Periodo:")
print(df_ref["Periodo"].value_counts())

print("\n=== CALCULANDO CONFORMIDADE POR CICLO ===")
print("Calculando media da posicao no ciclo por Mes x KM x Ciclo x Periodo...")

gb_cols = ["Mes", "KM", "Ciclo", "Periodo"]

df_ciclo = (
    df_ref
    .groupby(gb_cols, as_index=False)
    .agg(Media_Posicao_no_Ciclo=("Minuto_no_Ciclo", "mean"))
)

print(f"[OK] Tabela por ciclo criada: {len(df_ciclo):,} linhas")

print("Calculando saldo e status por ciclo...")
df_ciclo["Saldo_Ciclo"] = df_ciclo["Media_Posicao_no_Ciclo"] - LIMITE_INDIVIDUAL

df_ciclo["Status_Ciclo"] = np.where(
    df_ciclo["Media_Posicao_no_Ciclo"] <= LIMITE_INDIVIDUAL,
    "CONFORME",
    "NAO CONFORME",
)

print("\nResumo da conformidade por ciclo:")
print(df_ciclo["Status_Ciclo"].value_counts())
print(f"\nMedia da posicao no ciclo:")
print(df_ciclo["Media_Posicao_no_Ciclo"].describe())

print("\n=== CALCULANDO CONFORMIDADE POR PERIODO ===")
print("Calculando saldo por periodo (soma dos 4 ciclos)...")

gb_periodo = ["Mes", "KM", "Periodo"]

df_periodo = (
    df_ciclo
    .groupby(gb_periodo, as_index=False)
    .agg(Saldo_Periodo=("Saldo_Ciclo", "sum"))
)

print(f"[OK] Tabela por periodo criada: {len(df_periodo):,} linhas")

print("Definindo status do periodo com base no limite geral (<= 60 min)...")
df_periodo["Status_Periodo"] = np.where(
    df_periodo["Saldo_Periodo"] <= LIMITE_GERAL,
    "CONFORME",
    "NAO CONFORME",
)

print("\nResumo da conformidade por periodo:")
print(df_periodo["Status_Periodo"].value_counts())
print(f"\nSaldo do periodo:")
print(df_periodo["Saldo_Periodo"].describe())

print("\n=== SALVANDO TABELAS GOLD ===")
print("Salvando tabelas Gold em parquet...")

df_ciclo.to_parquet(ARQ_GOLD_CICLO, index=False)
if os.path.exists(ARQ_GOLD_CICLO):
    tamanho_mb = os.path.getsize(ARQ_GOLD_CICLO) / (1024 * 1024)
    print(f"[OK] Salvo: {ARQ_GOLD_CICLO} ({tamanho_mb:.2f} MB)")
else:
    print(f"[ERRO] Arquivo nao foi criado: {ARQ_GOLD_CICLO}")

df_periodo.to_parquet(ARQ_GOLD_PERIODO, index=False)
if os.path.exists(ARQ_GOLD_PERIODO):
    tamanho_mb = os.path.getsize(ARQ_GOLD_PERIODO) / (1024 * 1024)
    print(f"[OK] Salvo: {ARQ_GOLD_PERIODO} ({tamanho_mb:.2f} MB)")
else:
    print(f"[ERRO] Arquivo nao foi criado: {ARQ_GOLD_PERIODO}")

print("\nSalvando versoes Excel das tabelas Gold...")

try:
    with pd.ExcelWriter(ARQ_GOLD_CICLO_XLSX) as writer:
        df_ciclo.to_excel(writer, index=False, sheet_name="Conformidade_Ciclo_KM")
    if os.path.exists(ARQ_GOLD_CICLO_XLSX):
        tamanho_mb = os.path.getsize(ARQ_GOLD_CICLO_XLSX) / (1024 * 1024)
        print(f"[OK] Salvo: {ARQ_GOLD_CICLO_XLSX} ({tamanho_mb:.2f} MB)")
    else:
        print(f"[ERRO] Arquivo Excel nao foi criado: {ARQ_GOLD_CICLO_XLSX}")
except Exception as e:
    print(f"[AVISO] Nao foi possivel salvar Excel ciclo: {e}")

try:
    with pd.ExcelWriter(ARQ_GOLD_PERIODO_XLSX) as writer:
        df_periodo.to_excel(writer, index=False, sheet_name="Conformidade_Periodo_KM")
    if os.path.exists(ARQ_GOLD_PERIODO_XLSX):
        tamanho_mb = os.path.getsize(ARQ_GOLD_PERIODO_XLSX) / (1024 * 1024)
        print(f"[OK] Salvo: {ARQ_GOLD_PERIODO_XLSX} ({tamanho_mb:.2f} MB)")
    else:
        print(f"[ERRO] Arquivo Excel nao foi criado: {ARQ_GOLD_PERIODO_XLSX}")
except Exception as e:
    print(f"[AVISO] Nao foi possivel salvar Excel periodo: {e}")

print("\n=== CONCLUSAO ===")
print("Camada Gold criada com sucesso!")
print(f"\nArquivos gerados:")
print(f"  1. Parquet Ciclo: {ARQ_GOLD_CICLO}")
print(f"  2. Parquet Periodo: {ARQ_GOLD_PERIODO}")
print(f"  3. Excel Ciclo: {ARQ_GOLD_CICLO_XLSX}")
print(f"  4. Excel Periodo: {ARQ_GOLD_PERIODO_XLSX}")
print(f"\nResumo:")
print(f"  - Linhas na tabela por ciclo: {len(df_ciclo):,}")
print(f"  - Linhas na tabela por periodo: {len(df_periodo):,}")
print(f"\nProximo passo: Execute 04_gerar_painel_consolidado_mvp")

=== CONFIGURACAO DE CAMINHOS ===
Workspace base: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp
Diretorio Silver: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver
Diretorio Gold: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold

Arquivo silver referencia: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver/silver_ciclos_referencia.parquet
Saida Gold (ciclo): /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold/gold_conformidade_ciclo_km.parquet
Saida Gold (periodo): /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold/gold_conformidade_periodo_km.parquet
Saida Excel ciclo: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold/gold_conformidade_ciclo_km.xlsx
Saida Excel periodo: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold/gold_conformidade_periodo_km.xlsx

=== CONFIGURACAO DE LIMITES ===
Limite individual (ciclo): 180.0 minutos
Limite geral (periodo): 60.0 minutos

[OK] Diretorio Gold criado/verificado: /Workspace/Users/romul